In [11]:
!pip install --user -U nltk

import torch
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


[nltk_data] Downloading package punkt to /Users/sfilatov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sfilatov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
raw_text = open('../lolita.txt', 'rb').read()

In [42]:
from nltk.tokenize import RegexpTokenizer

SIZE = 200
sentences = []

sent_text = nltk.sent_tokenize(raw_text.decode("windows-1251"))
for sentence in sent_text:
    tokenized_text = nltk.word_tokenize(sentence)
#     tagged = nltk.pos_tag(tokenized_text)
    sentences.append(tokenized_text)

# skip intro
intro = 30
sentences = sentences[intro:SIZE+intro]

In [43]:
len(sentences)

200

In [44]:
min(len(s) for s in sentences)

2

In [45]:
max(len(s) for s in sentences)

121

In [46]:
res = set()
for s in sentences:
  for w in s:
    res.add(w)
    
res.add('<S>')
res.add('<E>')
L = len(res)

In [47]:
wtoi = {w:i for i,w in enumerate(list(res))}
itow = {i:w for w,i in wtoi.items()}

In [48]:
N = torch.zeros((L, L), dtype=torch.int32)

In [49]:
for s in sentences:
  ws = ['<S>'] + list(s) + ['<E>']
  for w1, w2 in zip(ws, ws[1:]):
    ix1 = wtoi[w1]
    ix2 = wtoi[w2]
    N[ix1, ix2] += 1
    
N

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [50]:
p = N[0].float()
p = p / p.sum()

In [51]:
g = torch.Generator().manual_seed(2147483647)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itow[ix]

'отметки'

In [52]:
P = torch.multinomial(p, num_samples=100, replacement=True, generator=g)
p.shape

torch.Size([2667])

In [59]:
P = (N+0.0001).float() # probs are too small
P /= P.sum(1, keepdims=True)

In [60]:
start = wtoi['<S>']

In [61]:
P[start]

tensor([4.9933e-07, 4.9933e-07, 4.9933e-07,  ..., 4.9933e-07, 4.9933e-07,
        4.9933e-07])

In [65]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    ix = start
    while True:
        p = P[ix]
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        w = itow[ix]
        if w == '<E>':
          break
        if w not in [',', '.', '?', '!']:
            out.append(' ')
        out.append(w)
    print(''.join(out))
    print('\n')

 Почкообразная стадия в черных предлогом ударяя арфам ( надо мной, « ракетс » описывает с приятным волнением превозмогает там в досягаемости взрослых парижан шестнадцатого округа.


 Она то роковое лето давал ей подарить все допытываюсь у бедняги не человеческую, мешковатых, в мире.


 В возрастных пределах между собой то и пространства, социальных работников, отнюдь не с поволокой, будучи законоуважающим трусом, вероятно, а именно когда минуло смотревшего сквозь которую сколько мне бы они вьются увы, который окружен широким умным европейским большую часть времени и кому какое дело поднимала горсть мелкого пляжного песочка и общую мораль.


 3 было, благороднокрылые серафимы… Полюбуйтесь-ка теоретический разговор о мои нимфетки.


 Итак суматоха зубы.




In [66]:
# create the training set of bigrams (x,y)
xs, ys = [], []

for s in sentences:
  ws = ['<S>'] + list(s) + ['<E>']
  for w1, w2 in zip(ws, ws[1:]):
    ix1 = wtoi[w1]
    ix2 = wtoi[w2]
    print(w1, w2)
    xs.append(ix1)
    ys.append(ix2)
    
xs = torch.tensor(xs)
ys = torch.tensor(ys)

<S> Мое
Мое задание
задание оказалось
оказалось проще
проще ,
, чем
чем мы
мы с
с ним
ним предполагали
предполагали .
. <E>
<S> Если
Если не
не считать
считать исправления
исправления явных
явных описок
описок да
да тщательного
тщательного изъятия
изъятия некоторых
некоторых цепких
цепких деталей
деталей ,
, которые
которые ,
, несмотря
несмотря на
на старания
старания самого
самого «
« Г
Г .
. <E>
<S> Г.
Г. »
» ,
, еще
еще уцелели
уцелели в
в тексте
тексте ,
, как
как некие
некие вехи
вехи и
и памятники
памятники (
( указатели
указатели мест
мест и
и людей
людей ,
, которых
которых приличие
приличие требовало
требовало обойти
обойти молчанием
молчанием ,
, а
а человеколюбие
человеколюбие –
– пощадить
пощадить )
) ,
, можно
можно считать
считать ,
, что
что эти
эти примечательные
примечательные записки
записки представлены
представлены в
в неприкосновенности
неприкосновенности .
. <E>
<S> Причудливый
Причудливый псевдоним
псевдоним их
их автора
автора –
– его
его собственное
собственно

жителях и
и природе
природе Америки
Америки смешны
смешны .
. <E>
<S> Отчаянная
Отчаянная честность
честность ,
, которой
которой трепещет
трепещет его
его исповедь
исповедь ,
, отнюдь
отнюдь не
не освобождает
освобождает его
его от
от ответственности
ответственности за
за дьявольскую
дьявольскую изощренность
изощренность .
. <E>
<S> Он
Он ненормален
ненормален .
. <E>
<S> Он
Он не
не джентльмен
джентльмен .
. <E>
<S> Но
Но с
с каким
каким волшебством
волшебством певучая
певучая его
его скрипка
скрипка возбуждает
возбуждает в
в нас
нас нежное
нежное сострадание
сострадание к
к Лолите
Лолите ,
, заставляя
заставляя нас
нас зачитываться
зачитываться книгой
книгой ,
, несмотря
несмотря на
на испытываемое
испытываемое нами
нами отвращение
отвращение к
к автору
автору !
! <E>
<S> Как
Как описание
описание клинического
клинического случая
случая ,
, «
« Лолите
Лолите »
» ,
, несомненно
несомненно ,
, суждено
суждено стать
стать одним
одним из
из классических
классических произведений
произве

, скользить
скользить на
на водяных
водяных лыжах
лыжах ,
, читал
читал мне
мне «
« Дон-Кихота
Дон-Кихота »
» и
и «
« Les
Les Misеrables
Misеrables »
» ,
, и
и я
я обожал
обожал и
и чтил
чтил его
его и
и радовался
радовался за
за него
него ,
, когда
когда случалось
случалось подслушать
подслушать ,
, как
как слуги
слуги обсуждают
обсуждают его
его разнообразных
разнообразных любовниц
любовниц –
– ласковых
ласковых красавиц
красавиц ,
, которые
которые очень
очень много
много мною
мною занимались
занимались ,
, воркуя
воркуя надо
надо мной
мной и
и проливая
проливая драгоценные
драгоценные слезы
слезы над
над моим
моим вполне
вполне веселым
веселым безматеринством
безматеринством .
. <E>
<S> Я
Я учился
учился в
в английской
английской школе
школе ,
, находившейся
находившейся в
в нескольких
нескольких километрах
километрах от
от дома
дома ;
; там
там я
я играл
играл в
в «
« ракетс
ракетс »
» и
и «
« файвс
файвс »
» (
( ударяя
ударяя мяч
мяч об
об стену
стену ракеткой
ракеткой или
или ла

трещина через
через всю
всю мою
мою жизнь
жизнь ?
? <E>
<S> Или
Или ,
, может
может быть
быть ,
, острое
острое мое
мое увлечение
увлечение этим
этим ребенком
ребенком было
было лишь
лишь первым
первым признаком
признаком врожденного
врожденного извращения
извращения ?
? <E>
<S> Когда
Когда стараюсь
стараюсь разобраться
разобраться в
в былых
былых желаниях
желаниях ,
, намерениях
намерениях ,
, действиях
действиях ,
, я
я поддаюсь
поддаюсь некоему
некоему обратному
обратному воображению
воображению ,
, питающему
питающему аналитическую
аналитическую способность
способность возможностями
возможностями безграничными
безграничными ,
, так
так что
что всякий
всякий представляющийся
представляющийся мне
мне прошлый
прошлый путь
путь делится
делится без
без конца
конца на
на развилины
развилины в
в одуряюще
одуряюще сложной
сложной перспективе
перспективе памяти
памяти .
. <E>
<S> Я
Я уверен
уверен все
все же
же ,
, что
что волшебным
волшебным и
и роковым
роковым образом
образом Лолита
Лолит

несравненно более
более зависящих
зависящих от
от пространственного
пространственного мира
мира единовременных
единовременных явлений
явлений ,
, чем
чем от
от невесомого
невесомого острова
острова завороженного
завороженного времени
времени ,
, где
где Лолита
Лолита играет
играет с
с ей
ей подобными
подобными .
. <E>
<S> Внутри
Внутри тех
тех же
же возрастных
возрастных границ
границ число
число настоящих
настоящих нимфеток
нимфеток гораздо
гораздо меньше
меньше числа
числа некрасивых
некрасивых ,
, или
или просто
просто «
« миленьких
миленьких »
» ,
, или
или даже
даже «
« смазливых
смазливых »
» ,
, но
но вполне
вполне заурядных
заурядных ,
, пухленьких
пухленьких ,
, мешковатых
мешковатых ,
, холоднокожих
холоднокожих ,
, человечьих
человечьих по
по природе
природе своей
своей девочек
девочек ,
, с
с круглыми
круглыми животиками
животиками ,
, с
с косичками
косичками ,
, таких
таких ,
, которые
которые могут
могут или
или не
не могут
могут потом
потом превратиться
превратиться в
в 

. <E>
<S> В
В конце
конце концов
концов ,
, Данте
Данте безумно
безумно влюбился
влюбился в
в свою
свою Беатриче
Беатриче ,
, когда
когда минуло
минуло только
только девять
девять лет
лет ей
ей ,
, такой
такой искрящейся
искрящейся ,
, крашеной
крашеной ,
, прелестной
прелестной ,
, в
в пунцовом
пунцовом платье
платье с
с дорогими
дорогими каменьями
каменьями ,
, а
а было
было это
это в
в 1274-ом
1274-ом году
году ,
, во
во Флоренции
Флоренции ,
, на
на частном
частном пиру
пиру ,
, в
в веселом
веселом мае
мае месяце
месяце .
. <E>
<S> Когда
Когда же
же Петрарка
Петрарка безумно
безумно влюбился
влюбился в
в свою
свою Лаурину
Лаурину ,
, она
она была
была белокурой
белокурой нимфеткой
нимфеткой двенадцати
двенадцати лет
лет ,
, бежавшей
бежавшей на
на ветру
ветру ,
, сквозь
сквозь пыль
пыль и
и цветень
цветень ,
, сама
сама как
как летящий
летящий цветок
цветок ,
, среди
среди прекрасной
прекрасной равнины
равнины ,
, видимой
видимой с
с Воклюзских
Воклюзских холмов
холмов .
. <E>
<S> 

In [67]:
xs

tensor([2269,  360, 1725,  ..., 1311,  426, 1076])

In [68]:
ys

tensor([ 360, 1725,  714,  ...,  426, 1076, 1979])

In [69]:
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((L, L), generator=g)

In [71]:
import torch.nn.functional as F

In [72]:
xenc = F.one_hot(xs, num_classes=L).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
# btw: the last 2 lines here are together called a 'softmax'

In [73]:
probs.shape

torch.Size([5754, 2667])

In [74]:

nlls = torch.zeros(5)
for i in range(5):
  # i-th bigram:
  x = xs[i].item() # input character index
  y = ys[i].item() # label character index
  print('--------')
  print(f'bigram example {i+1}: {itow[x]} {itow[y]} (indexes {x},{y})')
  print('input to the neural net:', x)
  print('output probabilities from the neural net:', probs[i])
  print('label (actual next character):', y)
  p = probs[i, y]
  print('probability assigned by the net to the the correct character:', p.item())
  logp = torch.log(p)
  print('log likelihood:', logp.item())
  nll = -logp
  print('negative log likelihood:', nll.item())
  nlls[i] = nll

print('=========')
print('average negative log likelihood, i.e. loss =', nlls.mean().item())

--------
bigram example 1: <S> Мое (indexes 2269,360)
input to the neural net: 2269
output probabilities from the neural net: tensor([2.8644e-04, 4.4135e-04, 1.5681e-03,  ..., 2.5529e-04, 3.4640e-04,
        6.5233e-05])
label (actual next character): 360
probability assigned by the net to the the correct character: 2.55229515460087e-05
log likelihood: -10.575932502746582
negative log likelihood: 10.575932502746582
--------
bigram example 2: Мое задание (indexes 360,1725)
input to the neural net: 360
output probabilities from the neural net: tensor([9.5134e-05, 7.4284e-05, 1.4969e-04,  ..., 6.1586e-04, 3.6249e-05,
        1.7072e-04])
label (actual next character): 1725
probability assigned by the net to the the correct character: 0.005446252413094044
log likelihood: -5.212827682495117
negative log likelihood: 5.212827682495117
--------
bigram example 3: задание оказалось (indexes 1725,714)
input to the neural net: 1725
output probabilities from the neural net: tensor([6.1580e-04, 7.97

In [75]:
# randomly initialize 27 neurons' weights. each neuron receives 27 inputs
g = torch.Generator().manual_seed(2147483647)
W = torch.randn((L, L), generator=g, requires_grad=True)

In [76]:
# forward pass

In [77]:
xenc = F.one_hot(xs, num_classes=L).float() # input to the network: one-hot encoding
logits = xenc @ W # predict log-counts
counts = logits.exp() # counts, equivalent to N
probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
loss = -probs[torch.arange(len(ys)), ys].log().mean()

In [78]:
print(loss.item())

8.4583158493042


In [79]:
# backward pass
W.grad = None # set to zero the gradient
loss.backward()

In [80]:
W.data += -0.1 * W.grad

In [81]:
# --------- !!! OPTIMIZATION !!! yay, but this time actually --------------

In [82]:
num = xs.nelement()
print('number of examples: ', num)

number of examples:  5754


In [89]:
# gradient descent
for k in range(50):
  
  # forward pass
  xenc = F.one_hot(xs, num_classes=L).float() # input to the network: one-hot encoding
  logits = xenc @ W # predict log-counts
  counts = logits.exp() # counts, equivalent to N
  probs = counts / counts.sum(1, keepdims=True) # probabilities for next character
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(k, loss.item())
  
  # backward pass
  W.grad = None # set to zero the gradient
  loss.backward()
  
  # update
  W.data += -400 * W.grad

0 2.9810469150543213
1 2.9526898860931396
2 2.9217185974121094
3 2.894425868988037
4 2.8645215034484863
5 2.838301420211792
6 2.809471607208252
7 2.784330368041992
8 2.7565784454345703
9 2.732518196105957
10 2.7058446407318115
11 2.6828646659851074
12 2.6572670936584473
13 2.635362386703491
14 2.6108343601226807
15 2.589996576309204
16 2.566527843475342
17 2.5467450618743896
18 2.5243215560913086
19 2.50557804107666
20 2.484182596206665
21 2.466459035873413
22 2.446070671081543
23 2.429344892501831
24 2.4099395275115967
25 2.3941855430603027
26 2.3757362365722656
27 2.360926389694214
28 2.3434040546417236
29 2.3295071125030518
30 2.3128790855407715
31 2.2998626232147217
32 2.2840962409973145
33 2.271925210952759
34 2.2569854259490967
35 2.24562406539917
36 2.231473922729492
37 2.220886468887329
38 2.207489252090454
39 2.1976373195648193
40 2.184955596923828
41 2.175802230834961
42 2.1637985706329346
43 2.155306100845337
44 2.1439433097839355
45 2.136075258255005
46 2.125316858291626
47

In [90]:
print(loss.item())

2.101118326187134


In [91]:
xenc = F.one_hot(torch.tensor([ix]), num_classes=L).float()
xenc.shape

torch.Size([1, 2667])

In [92]:
g = torch.Generator().manual_seed(2147483647)
xenc = F.one_hot(torch.tensor([ix]), num_classes=len(N)).float()
logits = xenc @ W 
counts = logits.exp()
p = counts / counts.sum(1, keepdims=True)
ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
itow[ix]

'сжигаем'

In [94]:
# finally, sample from the 'neural net' model
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
    out = []
    w = '<S>'
    ix = start
    while True:
        
        xenc = F.one_hot(torch.tensor([ix]), num_classes=L).float()
        logits = xenc @ W # predict log-counts
        counts = logits.exp() # counts, equivalent to N
        p = counts / counts.sum(1, keepdims=True)
        ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        w = itow[ix]
        if w == '<E>' or len(out) > 50:
          break
        if w not in [',', '.', '?', '!']:
            out.append(' ')
        out.append(w)
    print(''.join(out))
    print('\n')

 Почкообразная стадия слоновой убийцу в нашем чрезвычайно привязан нечто вроде пещеры, Позволю себе напомнить течение библиотеке.


 Она вздрагивала и уже ровня ; мне дорогие конфеты отсутствием путешествие ; но в нескольких шагах от него исходило собственное измышление космосом великолепная гостиница воспользовался фирмы


 До тринадцати маска кроме множества парижан шестнадцатого что, мы, проводил большую часть времени в сокровеннейшую стараюсь очарованном острове упомянутых сравнительными примерами из моих чресел.


 Старшая общую мораль запреты неотвязно – которые, паренек 3 к напечатанию в 11 каким прекрасно уживался как оглянусь на прогулке, одинокие ободрения, на той


 Он относился нами отвращение к девяноста в дом и шелковистости большой яркий рот перед припадком – по одному вихрем держать кол затейливости телесное Меня ровня ;


